In [18]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split
from aif360.sklearn.inprocessing import ExponentiatedGradientReduction
from sklearn.svm import LinearSVC

In [2]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset,predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attribute):
    features = [privileged_attribute] + unprivileged_attribute # We want to check the fairness level regarding the protected attribute "sex"

    # This is the object made of the original dataset
    aif_sex_dataset = BinaryLabelDataset( # Base class for all structured datasets with binary labels.
            df=dataset,
            favorable_label=favorable_label_v, # This means that a prediction is biased toward the privileged attribute if its value is 1 (True)
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute], # here we tell AIF that we want to check for predictions
        )

    # We do the same thing but with the predictions dataset
    aif_sex_pred = BinaryLabelDataset(
            df=predictions,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    sex_privileged_group = [{privileged_attribute: 1, **{attr: 0 for attr in unprivileged_attribute}}]
    sex_unprivileged_group = [{attr: 1, privileged_attribute: 0} for attr in unprivileged_attribute]

    # We provide the ClassificationMetric object with all the information needed:
    # aif_sex_dataset - The original test set
    # aif_sex_pred - A dataset containing the predictions of the model
    # sex_privileged_group - The privileged group
    # sex_unprivileged_group - The unprivileged group
    fairness_metrics = ClassificationMetric(dataset=aif_sex_dataset,
                            classified_dataset=aif_sex_pred,
                            unprivileged_groups=sex_unprivileged_group,
                            privileged_groups=sex_privileged_group)
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Average Odds Difference (AOD): {AOD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")

In [3]:
dataset_path = "trained_compas-score.csv"
df_raw = pd.read_csv(dataset_path) 

pd.set_option('display.max_columns', None)

In [4]:
X = df_raw.drop(columns="is_recid")
y = df_raw["is_recid"]

# Define four sets and apply the function
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, # 0.2 indicates a test set size of 20%
                                                    random_state=42)

In [5]:
sex_features = ["sex_Male", "sex_Female"]
race_features = ["race_Caucasian", "race_African-American", "race_Asian", "race_Other", "race_Native American", "race_Hispanic"]

race_privileged = "race_Caucasian"
race_unprivileged = ["race_African-American", "race_Asian", "race_Other", "race_Native American", "race_Hispanic"]
sex_privileged = "sex_Female"
sex_unprivileged = ["sex_Male"]

In [ ]:
# First, we train our decision tree regularly
dt_clf = DecisionTreeClassifier(random_state=42)

# We create an ExponentiatedGradientReduction object that acts on the trained model to change its internal behavior
processed_dt_clf = ExponentiatedGradientReduction(prot_attr=sex_features, estimator=dt_clf, constraints='DemographicParity', drop_prot_attr=False)

# Then we train again the same model updated with the reduction
fair_dt_clf = processed_dt_clf.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
fair_dt_predictions = fair_dt_clf.predict(X_test)

compute_performance_metrics(y_test,fair_dt_predictions,"Decision Tree")

In [ ]:
# Definisci la pipeline e dai un nome al classificatore
svm_pipeline = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))

svm_classifier = svm_pipeline.named_steps['linearsvc'].fit(X_train, y_train.values.ravel())

# Crea l'oggetto ExponentiatedGradientReduction
processed_svm = ExponentiatedGradientReduction(prot_attr=sex_features, estimator=svm_classifier, constraints='DemographicParity', drop_prot_attr=False)

# Fai l'addestramento passando sample_weight al classificatore nella pipeline
fair_svm = processed_svm.fit(X_train, y_train)

# Fai delle predizioni sui dati di test
svm_pred = fair_svm.predict(X_test)

# Calcola le metriche di performance
compute_performance_metrics(y_test, svm_pred, "SVM")

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

#rf_classifier.fit(X_train, y_train)

processed_rf = ExponentiatedGradientReduction(prot_attr=sex_features, estimator=svm_classifier, constraints='DemographicParity', drop_prot_attr=False)

fair_rf = processed_rf.fit(X_train, y_train)

rf_predictions = fair_rf.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

In [25]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_test.copy(deep=True) # we create a copy of the test set
dataset['is_recid'] = y_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['is_recid'] = fair_dt_predictions # but this time the target feature is made by the predictions of our model

In [26]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,sex_privileged,sex_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.024
Average Odds Difference (AOD): 0.016
Equal Opportunity Difference (EOD): 0.066


In [21]:
predictions['is_recid'] = svm_pred 

fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,sex_privileged,sex_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.016
Average Odds Difference (AOD): -0.011
Equal Opportunity Difference (EOD): 0.027


In [23]:
predictions['is_recid'] = rf_predictions 

fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,sex_privileged,sex_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.022
Average Odds Difference (AOD): -0.011
Equal Opportunity Difference (EOD): 0.012


In [ ]:
# First, we train our decision tree regularly
dt_clf = DecisionTreeClassifier(random_state=42)

# We create an ExponentiatedGradientReduction object that acts on the trained model to change its internal behavior
processed_dt_clf = ExponentiatedGradientReduction(prot_attr=race_features, estimator=dt_clf, constraints='DemographicParity', drop_prot_attr=False)

# Then we train again the same model updated with the reduction
fair_dt_clf = processed_dt_clf.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
fair_dt_predictions = fair_dt_clf.predict(X_test)

compute_performance_metrics(y_test,fair_dt_predictions,"Decision Tree")

In [ ]:
# Definisci la pipeline e dai un nome al classificatore
svm_pipeline = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))

svm_classifier = svm_pipeline.named_steps['linearsvc'].fit(X_train, y_train.values.ravel())

# Crea l'oggetto ExponentiatedGradientReduction
processed_svm = ExponentiatedGradientReduction(prot_attr=race_features, estimator=svm_classifier, constraints='DemographicParity', drop_prot_attr=False)

# Fai l'addestramento passando sample_weight al classificatore nella pipeline
fair_svm = processed_svm.fit(X_train, y_train)

# Fai delle predizioni sui dati di test
svm_pred = fair_svm.predict(X_test)

# Calcola le metriche di performance
compute_performance_metrics(y_test, svm_pred, "SVM")

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

#rf_classifier.fit(X_train, y_train)

processed_rf = ExponentiatedGradientReduction(prot_attr=race_features, estimator=svm_classifier, constraints='DemographicParity', drop_prot_attr=False)

fair_rf = processed_rf.fit(X_train, y_train)

rf_predictions = fair_rf.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

In [28]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_test.copy(deep=True) # we create a copy of the test set
dataset['is_recid'] = y_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task


In [29]:
predictions['is_recid'] = fair_dt_predictions # but this time the target feature is made by the predictions of our model

fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,race_privileged,race_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.02
Average Odds Difference (AOD): -0.011
Equal Opportunity Difference (EOD): 0.0


In [31]:
predictions['is_recid'] = svm_pred 

fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,race_privileged,race_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.031
Average Odds Difference (AOD): 0.001
Equal Opportunity Difference (EOD): -0.026


In [33]:
predictions['is_recid'] = rf_predictions 

fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,race_privileged,race_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.028
Average Odds Difference (AOD): 0.003
Equal Opportunity Difference (EOD): -0.019
